In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [21]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
epochs = 10

def get_data(training_data, test_data):
    return (
        DataLoader(training_data), #, shuffle=True),
        DataLoader(test_data),
    )

train_dl, valid_dl = get_data(training_data, test_data)

Using cpu device


In [48]:
x1,y1 = next(iter(train_dl))
x1, y1 =x1.flatten(), y1.squeeze()
x1.shape, y1.shape

(torch.Size([784]), torch.Size([]))

In [3]:
state_dict = torch.load('model.pth')
state_dict.keys()

odict_keys(['linear_relu_stack.0.weight', 'linear_relu_stack.0.bias', 'linear_relu_stack.2.weight', 'linear_relu_stack.2.bias'])

In [7]:
state_dict['linear_relu_stack.0.weight']

tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [-0.0215,  0.0106,  0.0308,  ..., -0.0199,  0.0161, -0.0342],
        [ 0.0350, -0.0297, -0.0037,  ...,  0.0171,  0.0238, -0.0001],
        [ 0.0085,  0.0223, -0.0324,  ..., -0.0296,  0.0182, -0.0296]])

In [9]:
weights = []
weights.append(state_dict['linear_relu_stack.0.weight'])
weights.append(state_dict['linear_relu_stack.2.weight'])
bias = []
bias.append(state_dict['linear_relu_stack.0.bias'])
bias.append(state_dict['linear_relu_stack.2.bias'])

In [109]:
def relu(X):
    X[X<0] = 0
    return X

def softmax(x):
    return x.exp() / x.exp().sum()
    
activation = []
activation.append(relu)
activation.append(softmax)
    
def forward(X):
    zs = [X]
    
    for w,b,a in zip(weights, bias, activation):
        zs.append(a(zs[-1]@w.T+b))
    return zs

def backward(y_pred,y):
    pass

In [110]:
zs = forward(x1)
[k.shape for k in zs]

[torch.Size([784]), torch.Size([18]), torch.Size([10])]

In [67]:
softmax(zs[-1])

tensor([-2.3650, -2.3650, -2.3650, -2.3650, -2.3650, -2.3650, -2.3583, -2.0818,
        -2.1453, -2.3025])